# Vectorizacion de texto

# MODELO 1.

In [1]:
#  TOPIC MODELING
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
import json
import glob
import re

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = pd.read_csv(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [2]:
def remove_stops(text, stops):
    text = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.) ]+','',text)
    text = re.sub(r'[^\w\s -]','',text)
    text = re.sub(r'-',' ',text)
    text = text.lower()
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

def clean_docs(docs):
    stops = stopwords.words("spanish")
    final = []
    for doc in docs:
        #clean_doc = remove_stops(doc, stops)
        final.append(doc)
    return (final)

In [22]:
encontrado = load_data("titulosEncontrados.csv")
candidato = load_data("titulosCandidatos.csv")
# print (descriptions[0])
candidato = candidato.dropna()
encontrado = encontrado.dropna()
titEncontrado = clean_docs(encontrado['titulo'])
titCandidato = clean_docs(candidato['titulo'])
# print (cleaned_docs[0])


In [25]:
dfTitCandidato = pd.read_csv('df_titulosCandidatos.csv', sep=',',  encoding='utf-8')
dfTitEncontrado = pd.read_csv('df_titulosEncontrados.csv', sep=',',  encoding='utf-8')

In [4]:
stops = stopwords.words("spanish")

vectorizer = TfidfVectorizer(
                                lowercase=False,
                                use_idf=True,
                                max_features=20000,
                                max_df=0.99,
                                min_df=50,
                                ngram_range = (1,3),
                                stop_words = stops
                                

                            )

vectorizer.fit(titEncontrado)
vectorsCandidatos = vectorizer.transform(titCandidato)
vectorsEncontrado = vectorizer.transform(titEncontrado)
feature_names = vectorizer.get_feature_names_out()




In [5]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(vectorsCandidatos, vectorsEncontrado)

In [6]:
dist.shape

(13413, 5390)

In [7]:
import seaborn as sns
import numpy as np

result = np.where(dist > 0.8)  #buscar como se utiliza


In [8]:
#sns.heatmap(htm)

In [9]:
dist

array([[0.        , 0.20939011, 0.        , ..., 0.        , 0.        ,
        0.24865957],
       [0.        , 0.        , 0.89997961, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.42124112, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.18236778]])

In [10]:
true_k = 20

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=100, n_init=1)

model.fit(vectorsEncontrado)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

with open ("clusters_results.txt", "w", encoding="utf-8") as f:
    for i in range(true_k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write (' %s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")

In [11]:
#terms

'remito conclusiones medicolegales'

## CountVectorizer

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

encontrado = load_data("titulosEncontrados.csv")['titulo']
coun_vect = CountVectorizer(ngram_range=(1,3),max_features = 25)
coun_vect.fit(titEncontrado)
count_candidatos = coun_vect.transform(titCandidato)
count_encontrados = coun_vect.transform(titEncontrado)
count_array = count_encontrados.toarray()
dfCountVec = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
dfCountVec

,actor,antecedentes,antecedentes autos,autos,clinica,conclusiones,consideraciones,consideraciones medico,consideraciones medico legales,contestacion,...,interes medico,legal,legales,medico,medico legal,medico legales,pericia,petitorio,puntos,puntos pericia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,1,1,0,1,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,1,0,...,0,0,1,1,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5386,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
print(coun_vect.vocabulary_)

{'antecedentes': 1, 'interes': 14, 'medico': 18, 'legal': 16, 'interes medico': 15, 'medico legal': 19, 'examen': 11, 'actor': 0, 'consideraciones': 6, 'legales': 17, 'consideraciones medico': 7, 'medico legales': 20, 'consideraciones medico legales': 8, 'conclusiones': 5, 'contestacion': 9, 'puntos': 23, 'pericia': 21, 'contestacion puntos': 10, 'puntos pericia': 24, 'petitorio': 22, 'autos': 3, 'historia': 12, 'clinica': 4, 'historia clinica': 13, 'antecedentes autos': 2}


In [15]:
print(coun_vect.get_feature_names_out())

['actor' 'antecedentes' 'antecedentes autos' 'autos' 'clinica'
 'conclusiones' 'consideraciones' 'consideraciones medico'
 'consideraciones medico legales' 'contestacion' 'contestacion puntos'
 'examen' 'historia' 'historia clinica' 'interes' 'interes medico' 'legal'
 'legales' 'medico' 'medico legal' 'medico legales' 'pericia' 'petitorio'
 'puntos' 'puntos pericia']


In [16]:
coun_vect.get_feature_names_out()

array(['actor', 'antecedentes', 'antecedentes autos', 'autos', 'clinica',
       'conclusiones', 'consideraciones', 'consideraciones medico',
       'consideraciones medico legales', 'contestacion',
       'contestacion puntos', 'examen', 'historia', 'historia clinica',
       'interes', 'interes medico', 'legal', 'legales', 'medico',
       'medico legal', 'medico legales', 'pericia', 'petitorio', 'puntos',
       'puntos pericia'], dtype=object)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(count_candidatos, count_encontrados)

In [18]:
dist.shape

(13413, 5390)

In [ ]:
candidatoEncontrado = []
for titulo in titEncontrado:
    for candidato in titCandidato:
        if titulo == candidato:
            candidatoEncontrado.append(candidato)

In [42]:
candidatoEncontrado = []
for i in range(len(dfTitEncontrado)):
    titEncontrado = dfTitEncontrado.titulo[i]
    for j in range(len(dfTitCandidato)):
        if titEncontrado == dfTitCandidato.titulo[j]:
            id = dfTitCandidato.id[j]
            tituloCandidato = dfTitCandidato.titulo[j]
            candidatoEncontrado.append((id, tituloCandidato))

In [50]:
df_candidatoEncontrado = pd.DataFrame(candidatoEncontrado, columns=['id','titulo'])

In [52]:
#df_candidatoEncontrado.to_csv('df_candidatoEncontrado.csv', index=False)
#df_candidatoEncontrado.to_excel('df_candidatoEncontrado.xlsx', index=False)